# Get alert history for all devices

In [1]:
import os
import sys
from mxnet.gluon import data as gdata
from mxnet.io import NDArrayIter
import pandas as pd
import numpy as np

def load_data_alerts(batch_size):
    train_data = np.array(pd.read_csv('./data/alert/train.csv').values)
    test_data = np.array(pd.read_csv('./data/alert/test.csv').values)
    
    num_workers = 0 if sys.platform.startswith('win32') else 4
    
    train_iter = gdata.DataLoader(train_data, batch_size, shuffle=True, num_workers=num_workers)
    
    test_iter = gdata.DataLoader(test_data, batch_size, shuffle=False, num_workers=num_workers)
    
    return train_iter, test_iter

In [2]:
import gluonbook as gb
from mxnet import autograd, nd

num_inputs = 4
num_outputs = 3
batch_size = 2

train_iter, test_iter = load_data_alerts(batch_size)

W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros(num_outputs)
params = [W, b]

for param in params:
    param.attach_grad()

In [3]:
def softmax(X):
    X_exp = nd.exp(X)
    partition = X_exp.sum(axis=1, keepdims=True)
    return X_exp / partition

In [4]:
def net(X):
    return softmax(nd.dot(X, W) + b)

In [5]:
def cross_entropy(y_hat, y):
    return - nd.pick(y_hat, y).log()

In [6]:
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1) == y.astype('float32')).mean().asscalar()

In [7]:
def evaluate_accuracy(data_iter, net):
    acc = 0.
    for batch in data_iter:
        X = batch[:, :num_inputs]
        y = batch[:, num_inputs]
        acc += accuracy(net(X), y)
    return acc / len(data_iter)

In [8]:
evaluate_accuracy(test_iter, net)

MXNetError: [17:08:22] c:\jenkins\workspace\mxnet-tag\mxnet\src\io\../operator/elemwise_op_common.h:133: Check failed: assign(&dattr, (*vec)[i]) Incompatible attr in node  at 1-th input: expected int64, got float32

In [ ]:
def SGD(params, learning_rate):
    for param in params:
        param[:] = param - learning_rate * param.grad

In [ ]:
from mxnet import autograd

num_epochs, learning_rate = 5, .01

for epoch in range(num_epochs):
    train_loss = 0.
    train_acc = 0.
    for batch in train_iter:
        with autograd.record():
            X = batch[:, :num_inputs]
            y = batch[:, num_inputs]
            y_hat = net(X)
            loss = cross_entropy(y_hat, y)
        loss.backward()
        
        SGD(params, learning_rate / batch_size)
        
        train_loss += loss.mean().asscalar()
        train_acc += accuracy(y_hat, y)
        
    test_acc = evaluate_accuracy(test_iter, net)

    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch + 1, train_loss/len(train_iter), train_acc/len(train_iter), test_acc))
